In [1]:
import scanpy as sc
import anndata as an
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl

sc.set_figure_params(dpi_save=300,color_map = 'viridis_r')
sc.settings.verbosity = 1
sc.logging.print_header()

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/data/person/lab/fengcz/conda/envs/squidpy/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-21 16:14:15.691444: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-21 16:14:16.426009: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-21 16:14:16.527167: E e

scanpy==1.9.3 anndata==0.9.2 umap==0.5.3 numpy==1.25.2 scipy==1.11.4 pandas==1.3.5 scikit-learn==1.2.1 statsmodels==0.14.0 python-igraph==0.10.2 pynndescent==0.5.7


In [3]:
def calculate_clone_prob(adata):
    # Inputs: cnv_matrix (spots x features), initial_clone_labels (spots), use_t = False

    cnv_matrix = adata.obsm['X_pca']
    initial_clone_labels = adata.obs['Spatial_clone'].values
    
    import numpy as np
    import pandas as pd
    from scipy.stats import multivariate_normal
    
    # initial_clone_labels: array of strings, e.g. ["Clone1","Clone2","Clone1",...]
    
    #  Encode string labels → numeric indices for array operations
    unique_labels = np.unique(initial_clone_labels)
    label_to_index = {lab: i for i, lab in enumerate(unique_labels)}
    index_to_label = {i: lab for lab, i in label_to_index.items()}
    
    clone_ids = np.array([label_to_index[lab] for lab in initial_clone_labels])
    K = len(unique_labels)
    
    #  Compute centroids per clone
    centroids = np.vstack([
        cnv_matrix[clone_ids == k].mean(axis=0)
        for k in range(K)
    ])
    
    #  Estimate covariance (pooled diagonal with ridge)
    residuals = np.vstack([
        cnv_matrix[clone_ids == k] - centroids[k]
        for k in range(K)
    ])
    cov = np.cov(residuals, rowvar=False)
    eps = 1e-6 * np.mean(np.diag(cov))
    cov_shrink = np.diag(np.diag(cov) + eps)
    
    #  Compute posterior probabilities
    log_likelihoods = np.zeros((cnv_matrix.shape[0], K))
    for k in range(K):
        log_likelihoods[:, k] = multivariate_normal.logpdf(
            cnv_matrix, mean=centroids[k], cov=cov_shrink
        )
    
    priors = np.array([(clone_ids == k).mean() for k in range(K)])
    log_post = log_likelihoods + np.log(priors + 1e-12)
    log_post -= log_post.max(axis=1, keepdims=True)
    post = np.exp(log_post)
    post = post / post.sum(axis=1, keepdims=True)
    
    # Probability for the clone originally assigned to each spot
    original_clone_prob = post[np.arange(len(clone_ids)), clone_ids]
    
    # Max probability across all clones (for comparison)
    top_probs = post.max(axis=1)
    
    # New assigned clone (highest posterior)
    assigned_numeric = post.argmax(axis=1)
    assigned_labels = np.array([index_to_label[i] for i in assigned_numeric])
    
    # Shannon entropy for uncertainty
    entropy = -np.sum(post * np.log(post + 1e-12), axis=1)


    adata.obs['initial_clone_label'] = initial_clone_labels
    adata.obs['assigned_clone'] = assigned_labels
    adata.obs['prob_initial_clone'] = original_clone_prob
    adata.obs['prob_assigned_clone'] = top_probs
    adata.obs['entropy'] = entropy
    adata.obs['prob_clone1'] = pd.DataFrame(post).iloc[:,label_to_index['clone_1']].values

    return(adata.obs)
    # return(post)


In [4]:
adata_006 = sc.read('./window100/P6_YF_1/clone_assigned.h5ad')
prob = calculate_clone_prob(adata_006)
prob.to_csv('posterior_probability/OV006.csv')